# House Price - Adcanced Regression Tchniques
![](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

# Load the House Price Data set from github

In [85]:
! wget -nc https://raw.githubusercontent.com/Jiansiyu/Classes/master/SYS6018_data%20mining/homework/homeworkproj/homework8/house-prices-advanced-regression-techniques/test.csv
! wget -nc https://raw.githubusercontent.com/Jiansiyu/Classes/master/SYS6018_data%20mining/homework/homeworkproj/homework8/house-prices-advanced-regression-techniques/train.csv
! wget -nc https://raw.githubusercontent.com/Jiansiyu/Classes/master/SYS6018_data%20mining/homework/homeworkproj/homework8/house-prices-advanced-regression-techniques/sample_submission.csv

File ‘test.csv’ already there; not retrieving.

File ‘train.csv’ already there; not retrieving.

File ‘sample_submission.csv’ already there; not retrieving.



# Load the dataset to the 

In [71]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge,LassoCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import  skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
%matplotlib inline

In [72]:
train = pd.read_csv("./train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Clean The Dataset

## data clean process

In [73]:
typoTable = {"Exterior2nd":[{"Brk Cmn":"BrkComm"},{"Wd Shng":"Wd Sdng"},{"CmentBd":"CemntBd"}]
             }

def typoCorrection(df):
    '''
    used correct the typos in the feature columnn according to the typotable
    :param df:
    :return:
    '''
    for colName, replaceTerm in typoTable.items():
        for replaceKey in replaceTerm:
          df[colName] = df[colName].replace(replaceKey)
    return df

def fillnumerical(df):
    '''

    :param df:
    :return:
    '''

    # size = len(df)
    # for colname in df:
    #     nanSum = df[colname].isnull().sum()
    #     col_type = df[colname].dtypes == "object"
    #     if nanSum > 0 and not col_type:
    #       if nanSum/size < 0.85:
    #         df[colname].fillna( df[colname].median(), inplace=True)
    #       else:
    #         df[colname].fillna( df[colname].mean(), inplace=True)
    return df


def fillCat(features):
    '''

    :param features:
    :return:
    '''
    features['Functional'] = features['Functional'].fillna('Typ')
    features['Electrical'] = features['Electrical'].fillna("SBrkr")
    features['KitchenQual'] = features['KitchenQual'].fillna("TA")
    features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
    features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
    features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

    features["PoolQC"] = features["PoolQC"].fillna("None")

    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        features[col] = features[col].fillna(0)
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        features[col] = features[col].fillna('None')
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        features[col] = features[col].fillna('None')

    features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

    objects = []
    for i in features.columns:
        if features[i].dtype == object:
            objects.append(i)

    features.update(features[objects].fillna('None'))

    features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

    # Filling in the rest of the NA's

    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numerics.append(i)
    features.update(features[numerics].fillna(0))

    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerics2 = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numerics2.append(i)

    skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

    high_skew = skew_features[skew_features > 0.5]
    skew_index = high_skew.index

    for i in skew_index:
        features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

    return features

def featureGenerator(features):
    '''
    :param df:
    :return:
    : dis : generate new features
    '''
    #features = features.drop(['Utilities', 'Street', 'PoolQC',], axis=1)

    features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']

    features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

    features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                     features['1stFlrSF'] + features['2ndFlrSF'])

    features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                                   features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))

    features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                                  features['EnclosedPorch'] + features['ScreenPorch'] +
                                  features['WoodDeckSF'])

    # simplified features
    features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
    return features

def simplifyer(df):

    return df

def featurize(features):
    '''
    Convert some number feature to class feature
    :param features:
    :return:  the transformed new dataframe
    '''
    features['MSSubClass'] = features['MSSubClass'].astype(str)
    features['YrSold'] = features['YrSold'].astype(str)
    features['MoSold'] = features['MoSold'].astype(str)
    return features
def dropFeatues(df):
    '''

    :param df:
    :return:
    '''
    return  df

def dummirize(df):
    dummied = pd.get_dummies(df).reset_index(drop=True)
    return dummied


def cleanData(df):
    '''
    :param df:
    :return:
    '''
    # apply typo correction
    df = typoCorrection(df)
    df = featurize(df)
    # df = fillnumerical(df)
    df = fillCat(df)
    df = featureGenerator(df)
    df = dropFeatues(df)
    df = dummirize(df)
    return df

In [74]:
def Load_DataSet():
    baseDir = "./"
    train_name = "train.csv"
    test_name  = "test.csv"

    train_file = os.path.join(baseDir, train_name)
    test_file   = os.path.join(baseDir,test_name)

    train_pd = pd.read_csv(train_file,index_col="Id")
    test_pd  = pd.read_csv(test_file,index_col= "Id")
    train_pd = train_pd[train_pd.GrLivArea < 4500]
    train_pd.reset_index(drop=True, inplace=True)
    train_pd["SalePrice"] = np.log1p(train_pd["SalePrice"])

    train_feature = train_pd.drop(["SalePrice"],axis = 1)
    trainPrice = train_pd.SalePrice.reset_index(drop = True)
    test_feature = test_pd

    print(train_feature.shape, test_feature.shape)
    # merge the data together for pre-processing
    df = pd.concat([train_feature,test_feature]).reset_index(drop = True)
    #TODO preprocess the dataset

    df = cleanData(df)

    train_pd = df.iloc[:len(train_pd),:]
    test_pd  = df.iloc[len(train_pd):, :]
    # train_pd= df.loc[train_pd.index, :]
    # test_pd = df.loc[test_pd.index, :]

    # splite the pre-preocessed dataset
    return train_pd, trainPrice, test_pd

# Mutal Information

In [75]:
from sklearn.feature_selection import mutual_info_regression
import matplotlib.pyplot as plt

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [76]:
train,trainPrice, test = Load_DataSet()
train.head()
X = train.copy()
y = trainPrice
test
# #label encoding for categoricals
# for colname in X.select_dtypes("object"):
#     X[colname], _ = X[colname].factorize()
# #discrete features should have integer dtypes
# discrete_features = X.dtypes == int
#
# mi_scores = make_mi_scores(X, y, discrete_features)
# # mi_scores
# plt.figure(dpi = 100, figsize = (8,15))
# # plot_mi_scores(mi_scores)
#


(1458, 79) (1459, 79)


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1458,20.673625,14.550687,5,4.679501,1961,1961,0.000000,107.505912,7.760721,42.813715,...,0,0,0,1,0,0,0,0,1,0
1459,20.834996,15.023149,6,4.679501,1958,1958,14.642918,174.393644,0.000000,54.675311,...,0,0,0,1,0,0,0,0,1,0
1460,19.687482,14.950918,5,3.991517,1997,1998,0.000000,156.263694,0.000000,28.375987,...,0,0,0,1,0,0,0,0,1,0
1461,20.348384,14.204772,6,4.679501,1998,1998,6.157403,128.653472,0.000000,47.769042,...,0,0,0,1,0,0,0,0,1,0
1462,13.949261,12.696875,8,3.991517,1992,1992,0.000000,71.179547,0.000000,94.312871,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,8.726308,10.765922,4,5.348041,1970,1970,0.000000,0.000000,0.000000,65.236276,...,0,0,0,1,0,0,0,0,1,0
2913,8.726308,10.723223,4,3.991517,1970,1970,0.000000,69.030589,0.000000,45.062871,...,0,0,0,1,1,0,0,0,0,0
2914,31.755371,15.820339,5,5.348041,1960,1996,0.000000,213.123315,0.000000,0.000000,...,0,0,0,1,1,0,0,0,0,0
2915,17.611139,14.307159,5,3.991517,1992,1992,0.000000,85.016479,0.000000,67.275011,...,0,0,0,1,0,0,0,0,1,0


# Training process

## Prepare the Data Set used for the training and test

In [77]:
import warnings
warnings.filterwarnings('ignore')

## overwrite the previous test set
X,y, test = Load_DataSet()

print("Train Feature : {}, price {}, test feature {}".format(X.shape, y.shape, test.shape))

for item in X.columns:
    print(item)

outliers = [30, 88, 462, 631, 1322]
X = X.drop(X.index[outliers])
y = y.drop(y.index[outliers])

overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.94:
        overfit.append(i)

overfit = list(overfit)
overfit.append('MSZoning_C (all)')

X = X.drop(overfit, axis=1).copy()

X_sub = test.drop(overfit, axis=1).copy()

print('X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)

(1458, 79) (1459, 79)
Train Feature : (1458, 341), price (1458,), test feature (1459, 341)
LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
YrBltAndRemod
TotalSF
Total_sqr_footage
Total_Bathrooms
Total_porch_sf
haspool
has2ndfloor
hasgarage
hasbsmt
hasfireplace
MSSubClass_120
MSSubClass_150
MSSubClass_160
MSSubClass_180
MSSubClass_190
MSSubClass_20
MSSubClass_30
MSSubClass_40
MSSubClass_45
MSSubClass_50
MSSubClass_60
MSSubClass_70
MSSubClass_75
MSSubClass_80
MSSubClass_85
MSSubClass_90
MSZoning_C (all)
MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Street_Grvl
Street_Pave
Alley_Grvl
Alley_None
Alley_Pave
LotShape_IR1
LotShape_IR2
LotShape_IR3
LotShape_Reg
LandContour_Bnk
L

## start the pipline and training the model

### predefine the constant

In [78]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import  KFold, cross_val_score
from  sklearn.linear_model import  ElasticNetCV, LassoCV, RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from datetime import  datetime


print('START ML', datetime.now(), )

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)


# rmsle
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


# build our model scoring function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y,
                                    scoring="neg_mean_squared_error",
                                    cv=kfolds))
    return (rmse)


# setup models
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds,))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=kfolds, random_state=42, l1_ratio=e_l1ratio))

svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =42)


lightgbm = LGBMRegressor(objective='regression',
                                       num_leaves=4,
                                       learning_rate=0.01,
                                       n_estimators=5000,
                                       max_bin=200,
                                       bagging_fraction=0.75,
                                       bagging_freq=5,
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )


xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006, random_state=42)

# stack
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                            gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)


print('TEST score on CV')

score = cv_rmse(ridge)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )


# # build our model scoring function
# def cv_rmse(model, X=X):
#     rmse = np.sqrt(-cross_val_score(model, X, y,
#                                     scoring="neg_mean_squared_error",
#                                     cv=kfolds))
#     return (rmse)
#
# # setup models
# alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
# alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
# e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
# e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
#
# kfolds = KFold(n_splits=10, shuffle=True, random_state= 10)
#
# ridgeReg = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt,cv = kfolds,))
#
# lassoReg = make_pipeline(RobustScaler(),LassoCV(max_iter=1e7, alphas=alphas2, random_state = 10, cv = kfolds))
#
# elasticNet = make_pipeline(RobustScaler(),
#                            ElasticNetCV(max_iter= 1e7, alphas=e_alphas,
#                                         cv = kfolds, random_state=10, l1_ratio=e_l1ratio)
#                            )
#
# svr = make_pipeline(RobustScaler(),
#                       SVR(C= 20, epsilon= 0.008, gamma=0.0003,))
#
#
# gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
#                                    max_depth=4, max_features='sqrt',
#                                    min_samples_leaf=15, min_samples_split=10,
#                                    loss='huber', random_state =42)
#
# lightgbm = LGBMRegressor(objective='regression',
#                                        num_leaves=4,
#                                        learning_rate=0.01,
#                                        n_estimators=5000,
#                                        max_bin=200,
#                                        bagging_fraction=0.75,
#                                        bagging_freq=5,
#                                        bagging_seed=7,
#                                        feature_fraction=0.2,
#                                        feature_fraction_seed=7,
#                                        verbose=-1,
#                                        #min_data_in_leaf=2,
#                                        #min_sum_hessian_in_leaf=11
#                                        )
#
#
# xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
#                                      max_depth=3, min_child_weight=0,
#                                      gamma=0, subsample=0.7,
#                                      colsample_bytree=0.7,
#                                      objective='reg:linear', nthread=-1,
#                                      scale_pos_weight=1, seed=27,
#                                      reg_alpha=0.00006, random_state=10)
#
# stack_gen = StackingCVRegressor(regressors=(ridgeReg, lassoReg, elasticNet,
#                                             gbr, xgboost, lightgbm),
#                                 meta_regressor=xgboost,
#                                 use_features_in_secondary=True)
#
#
# score = cv_rmse(ridgeReg)
# print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# #
# score = cv_rmse(lassoReg)
# print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
#
# score = cv_rmse(elasticNet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
#

START ML 2021-04-18 23:40:46.593612
TEST score on CV
Kernel Ridge score: 0.1023 (0.0143)
 2021-04-18 23:41:10.395239
Lasso score: 0.1032 (0.0147)
 2021-04-18 23:41:23.883886
ElasticNet score: 0.1033 (0.0148)
 2021-04-18 23:42:29.139108
SVR score: 0.1022 (0.0134)
 2021-04-18 23:42:32.171521


In [79]:
score = cv_rmse(svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lightgbm)
print("Lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()) )

score = cv_rmse(gbr)
print("GradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(xgboost)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

SVR score: 0.1022 (0.0134)

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction 

In [80]:
# rmsle
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


print('START Fit')
print(datetime.now(), 'StackingCVRegressor')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))
print(datetime.now(), 'elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)
print(datetime.now(), 'lasso')
lasso_model_full_data = lasso.fit(X, y)
print(datetime.now(), 'ridge')
ridge_model_full_data = ridge.fit(X, y)
print(datetime.now(), 'svr')
svr_model_full_data = svr.fit(X, y)
print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)
print(datetime.now(), 'xgboost')
xgb_model_full_data = xgboost.fit(X, y)
print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

START Fit
2021-04-18 23:45:54.282486 StackingCVRegressor
[23:47:23] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:27] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:32] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:37] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:42] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.

In [81]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.05 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))

print('RMSLE score on train data:')
print(rmsle(y, blend_models_predict(X)))

RMSLE score on train data:
0.05726731111469102


In [82]:
X.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,18.144573,13.833054,7,3.991517,2003,2003,19.433175,144.117862,0.0,29.991055,...,0,0,0,1,0,0,0,0,1,0
1,20.673625,14.117918,6,6.000033,1976,1976,0.000000,181.719186,0.0,44.135415,...,0,0,0,1,0,0,0,0,1,0
2,18.668047,14.476512,7,3.991517,2001,2002,17.768840,110.441033,0.0,56.896536,...,0,0,0,1,0,0,0,0,1,0
3,17.249651,14.106196,7,3.991517,1915,1970,0.000000,61.795315,0.0,64.808858,...,0,0,0,1,1,0,0,0,0,0
4,21.314283,15.022008,8,3.991517,2000,2000,25.404164,136.624601,0.0,61.166379,...,0,0,0,1,0,0,0,0,1,0


In [83]:
test.head()

subRes = test.copy()
subRes['SalePrice'] = np.expm1(blend_models_predict(X_sub))
subRes = subRes[['SalePrice']]
#subRes.index.name = "Id"
subRes#.to_csv("submit.csv")

,SalePrice
1458,123714.553983
1459,161158.571795
1460,185610.820026
1461,200275.191853
1462,188687.388831
...,...
2912,84568.520275
2913,80792.467215
2914,163537.036621
2915,115806.994124


In [84]:
submission = pd.read_csv("./sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(X_sub)))

submission.to_csv("submission_v5.csv",index=False)